In [ ]:
import os
import json
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from requests import Session, Request
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

# API Request Configuration
URL = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
PARAMETERS = {'start': '1', 'limit': '15', 'convert': 'USD'}
HEADERS = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': '4613d2f4-2b10-46be-a5e3-8226d6ac3757'  # Replace with your actual API key
}

# Function to fetch API data
def fetch_crypto_data():
    session = Session()
    session.headers.update(HEADERS)
    try:
        response = session.get(URL, params=PARAMETERS)
        data = json.loads(response.text)
        return pd.json_normalize(data['data'])
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(f"API Error: {e}")
        return None

# Logging function
def log_message(message):
    with open("crypto_log.txt", "a") as log_file:
        log_file.write(f"{pd.to_datetime('now')}: {message}\n")

# Automated API calls with logging (THIS IS WHAT YOU WILL WANT TO CHANGE FOR YOUR OWN PERSONAL USE, CURRENTLY SET TO RUN EVERY HOUR)
MAX_REQUESTS = 150
request_count = 0

while request_count < MAX_REQUESTS:
    df = fetch_crypto_data()
    if df is not None:
        df['timestamp'] = pd.to_datetime('now')
        df.to_csv('crypto_data.csv', mode='a', index=False, header=not os.path.isfile('crypto_data.csv'))
        log_message(f"Request {request_count + 1}: Data fetched & saved successfully.")
        print(f"Request {request_count + 1}: Data fetched & saved at", pd.to_datetime('now'))
    
    request_count += 1
    time.sleep(3600)  # Wait for an hour before the next API call

# Data transformation for visualization
df_grouped = df.groupby('name', sort=False)[[
    'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d',
    'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d'
]].mean()

df_melted = df_grouped.stack().reset_index()
df_melted.columns = ['name', 'percent_change', 'values']
df_melted['percent_change'] = df_melted['percent_change'].str.replace(
    'quote.USD.percent_change_', '', regex=True)

# Plot data
sns.catplot(x='percent_change', y='values', hue='name', data=df_melted, kind='point')
plt.show()

# Bitcoin-specific data
df_bitcoin = df.query("name == 'Bitcoin'")[['name', 'quote.USD.price', 'timestamp']]
print(df_bitcoin)


Request 1: Data fetched & saved at 2025-03-24 13:26:55.353923
